# Data Cleaning

If you're coming into this without much background and want to get an understanding of what data is being used, take a look in `../data/raw/data_description.txt`. There you will find each feature and how it is described within the datasets. As you might find, there are many categorical data types within the datasets. This data will be handled during the cleaning process. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("mode.copy_on_write", True)

In [2]:
data_dir = "../data/raw/"

training = pd.read_csv(data_dir + "train.csv")
testing = pd.read_csv(data_dir + "test.csv")

Let's focus on the training set. We will perform cleaning on the fly with the exploration.

In [3]:
training.shape

(1460, 81)

Right away, this is quite a small dataset. It's also evident that there features missing many entires. How these features are delt with is going to be circumstantial. Referencing the `data_description.txt` file, we get a lot of answers. Many categorial features have options to be set to NA or None which are both handled as null values. Other features, like LotFrontage, simply do not contain an entry (which should be 0) if not connected to the street. Let's take care of those now. 

Also, since the training set is small, both sets will be combined together and a train/test split will be established when working with the model.

In [4]:
df = pd.concat([training, testing])
df.reset_index(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2919 non-null   int64  
 1   Id             2919 non-null   int64  
 2   MSSubClass     2919 non-null   int64  
 3   MSZoning       2915 non-null   object 
 4   LotFrontage    2433 non-null   float64
 5   LotArea        2919 non-null   int64  
 6   Street         2919 non-null   object 
 7   Alley          198 non-null    object 
 8   LotShape       2919 non-null   object 
 9   LandContour    2919 non-null   object 
 10  Utilities      2917 non-null   object 
 11  LotConfig      2919 non-null   object 
 12  LandSlope      2919 non-null   object 
 13  Neighborhood   2919 non-null   object 
 14  Condition1     2919 non-null   object 
 15  Condition2     2919 non-null   object 
 16  BldgType       2919 non-null   object 
 17  HouseStyle     2919 non-null   object 
 18  OverallQ

In [6]:
# MSZoning contains different zoning classifications. Rather than guessing, let's drop the missing rows.

df = df.dropna(subset=["MSZoning"])
df.shape # Proving the rows have been remove

(2915, 82)

In [7]:
# Filling the missing LotFrontage feautre values with the median of the column

df["LotFrontage"].fillna(0, inplace=True)
median = df["LotFrontage"].median()
df.fillna({"LotFrontage": median}, inplace=True)
df["LotFrontage"].info() # Should provide no missing entries

<class 'pandas.core.series.Series'>
Index: 2915 entries, 0 to 2918
Series name: LotFrontage
Non-Null Count  Dtype  
--------------  -----  
2915 non-null   float64
dtypes: float64(1)
memory usage: 45.5 KB


In [8]:
df.fillna({"Alley":"NA"}, inplace=True)
df["Alley"].info()

<class 'pandas.core.series.Series'>
Index: 2915 entries, 0 to 2918
Series name: Alley
Non-Null Count  Dtype 
--------------  ----- 
2915 non-null   object
dtypes: object(1)
memory usage: 45.5+ KB


In [9]:
print(df["Utilities"].isna().value_counts())
print(df["Utilities"].value_counts())

Utilities
False    2914
True        1
Name: count, dtype: int64
Utilities
AllPub    2913
NoSeWa       1
Name: count, dtype: int64


After dropping some rows there is only one missing value within Utilities. All but one of the Utilities feature values is 'AllPub". Therefore I'm going to give the single missing feature entry the "AllPub" value also.

In [10]:
# Filling missing value
df.fillna({"Utilities":"AllPub"}, inplace=True)
df["Utilities"].isna().value_counts()

Utilities
False    2915
Name: count, dtype: int64

In [11]:
# Check to see which row(s) are affected
print(df[["Exterior1st", "Exterior2nd"]][df["Exterior1st"].isna() | df["Exterior2nd"].isna()])
print(df[["Exterior1st", "Exterior2nd"]].value_counts())

     Exterior1st Exterior2nd
2151         NaN         NaN
Exterior1st  Exterior2nd
VinylSd      VinylSd        1005
MetalSd      MetalSd         437
HdBoard      HdBoard         383
Wd Sdng      Wd Sdng         351
Plywood      Plywood         186
                            ... 
Stone        HdBoard           1
             Stone             1
Stucco       BrkFace           1
             CmentBd           1
Plywood      CBlock            1
Name: count, Length: 84, dtype: int64


The most common value for both Exterior1st and Exterior2nd is VinyalSd. Filling those missing values with those entries would be a suitable option here.

In [12]:
# Fill missing values and verify they've been replaced
df.fillna({"Exterior1st":"VinylSd", "Exterior2nd": "VinylSd"}, inplace=True)
print(df[["Exterior1st", "Exterior2nd"]].isna().value_counts())

Exterior1st  Exterior2nd
False        False          2915
Name: count, dtype: int64


In [13]:
# See how many MasVnrArea feature values do not match the value of MasVnrType by getting the indicies to replace values (if missing, MasVnrArea should be 0).
query = df[df["MasVnrType"].isna() & df["MasVnrArea"].gt(0)]
df.loc[query.index, "MasVnrArea"] = 0

df.fillna({"MasVnrType": "None"}, inplace=True)
df.fillna({"MasVnrArea": 0}, inplace=True)

# Verify changes
df[["MasVnrType", "MasVnrArea"]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2915 entries, 0 to 2918
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MasVnrType  2915 non-null   object 
 1   MasVnrArea  2915 non-null   float64
dtypes: float64(1), object(1)
memory usage: 132.9+ KB


The choice was made to set any other values for MasVnrArea that were missing to 0 since they matched MasVnrType also missing. Rather than dropping the rows, this provides more data to work with.

In [14]:
query = df[df["BsmtQual"].isna() & df["BsmtCond"].isna()]
df.fillna({"BsmtQual": "NA","BsmtCond": "NA"}, inplace=True)

df[["BsmtQual", "BsmtCond"]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2915 entries, 0 to 2918
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   BsmtQual  2915 non-null   object
 1   BsmtCond  2915 non-null   object
dtypes: object(2)
memory usage: 132.9+ KB


In [15]:
query = df[df["BsmtExposure"].isna() & ((df["BsmtCond"] == "NA") | (df["BsmtQual"] == "NA"))]
df.loc[query.index, ["BsmtExposure"]] = "NA"

In [16]:
# Use the results found here to determine how missing BsmtExposure feature should be filled
df[["BsmtExposure", "BsmtQual", "BsmtCond"]][df["BsmtExposure"].isna() & ((df["BsmtCond"] != "NA") | (df["BsmtQual"] != "NA"))]

,BsmtExposure,BsmtQual,BsmtCond
948,NaN,Gd,TA
1487,NaN,Gd,TA
2348,NaN,Gd,TA


In [17]:
# Most common value is No, that is how the missing features will be filled.
df["BsmtExposure"][(df["BsmtQual"] == "Gd") & (df["BsmtCond"] == "TA")].value_counts()

BsmtExposure
No    655
Av    247
Gd    125
Mn    103
Name: count, dtype: int64

In [18]:
df.fillna({"BsmtExposure": "No"}, inplace=True)
df["BsmtExposure"].info()

<class 'pandas.core.series.Series'>
Index: 2915 entries, 0 to 2918
Series name: BsmtExposure
Non-Null Count  Dtype 
--------------  ----- 
2915 non-null   object
dtypes: object(1)
memory usage: 110.1+ KB


In [19]:
query = df[(df["BsmtFinType1"].isna()) & (df["BsmtQual"] == "NA") & (df["BsmtCond"] == "NA")]
print(df["BsmtFinType1"].isna().value_counts())
print(f"# of values that can be filled: {query.shape[0]}")
df.loc[query.index, ["BsmtFinType1"]] = "NA"


query = df[(df["BsmtFinType2"].isna()) & (df["BsmtQual"] == "NA") & (df["BsmtCond"] == "NA")]
print(df["BsmtFinType2"].isna().value_counts())
print(f"# of values that can be filled: {query.shape[0]}")
df.loc[query.index, ["BsmtFinType2"]] = "NA"

BsmtFinType1
False    2839
True       76
Name: count, dtype: int64
# of values that can be filled: 76
BsmtFinType2
False    2838
True       77
Name: count, dtype: int64
# of values that can be filled: 76


In [20]:
query = df[["BsmtFinType2", "BsmtCond", "BsmtQual"]][df["BsmtFinType2"].isna()]
print(query)
print(df["BsmtFinType2"][(df["BsmtCond"] == "TA") & (df["BsmtQual"] == "Gd")].value_counts())
df.loc[query.index, ["BsmtFinType2"]] = "Unf"

    BsmtFinType2 BsmtCond BsmtQual
332          NaN       TA       Gd
BsmtFinType2
Unf    1031
LwQ      23
ALQ      21
GLQ      20
Rec      19
BLQ      18
Name: count, dtype: int64


In [21]:
# Verify holes have been filled
df[["BsmtFinType1","BsmtFinType2"]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2915 entries, 0 to 2918
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   BsmtFinType1  2915 non-null   object
 1   BsmtFinType2  2915 non-null   object
dtypes: object(2)
memory usage: 132.9+ KB


In [22]:
df[["BsmtCond", "BsmtQual", "BsmtExposure"]][df["TotalBsmtSF"].isna() | df["BsmtFinSF1"].isna() | df["BsmtFinSF2"].isna() | df["BsmtUnfSF"].isna() | df["TotalBsmtSF"].isna()]

,BsmtCond,BsmtQual,BsmtExposure
2120,NA,NA,NA


In [23]:
df.fillna({"BsmtFinSF1" : 0, "BsmtFinSF2": 0, "BsmtUnfSF": 0, "TotalBsmtSF": 0}, inplace=True)

In [24]:
# Since this would be difficult to fill, just dropping the single missing row
query = df[df["Electrical"].isna()]
df.drop(query.index, inplace=True)

In [25]:
query = df[["BsmtCond", "BsmtQual", "BsmtExposure", "BsmtFullBath", "BsmtHalfBath"]][df["BsmtFullBath"].isna() | df["BsmtHalfBath"].isna()]
query

,BsmtCond,BsmtQual,BsmtExposure,BsmtFullBath,BsmtHalfBath
2120,NA,NA,NA,NaN,NaN
2188,NA,NA,NA,NaN,NaN


In [26]:
# Since both these rows do not contain basements, setting the values to 0
df.fillna({"BsmtFullBath": 0, "BsmtHalfBath": 0}, inplace=True)

In [27]:
# Check number of kitchens
print(df[["KitchenQual", "KitchenAbvGr"]][df["KitchenQual"].isna()])
print(df["KitchenQual"].value_counts())

     KitchenQual  KitchenAbvGr
1555         NaN             1
KitchenQual
TA    1489
Gd    1150
Ex     205
Fa      69
Name: count, dtype: int64


In [28]:
df.fillna({"KitchenQual": "TA"}, inplace=True)

In [29]:
df.fillna({"Functional": "Typ"}, inplace=True)

In [30]:
query = df[(df["Fireplaces"] == 0) & (df["FireplaceQu"].isna())]
df.fillna({"FireplaceQu": "NA"}, inplace=True)

In [31]:
df.fillna({"GarageType": "NA"}, inplace=True)

# Chose to drop these rows
query = df[(df["GarageType"] != "NA") & (df["GarageYrBlt"].isna() | df["GarageFinish"].isna())]
df.drop(index=query.index, inplace=True)

df.fillna({"GarageYrBlt": 0, "GarageFinish": "NA", "GarageQual": "NA", "GarageCond": "NA"}, inplace=True)

In [32]:
query = df[df["PoolArea"].eq(0) & df["PoolQC"].isna()]
df.loc[query.index, "PoolQC"] = "NA"

# Dropping the other pools becuase not enough information to fill those missing feature values
df.drop(index=df[df["PoolArea"].gt(0) & df["PoolQC"].isna()].index, inplace=True)

In [33]:
df.fillna({"Fence": "NA", "MiscFeature": "NA"}, inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2909 entries, 0 to 2918
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          2909 non-null   int64  
 1   Id             2909 non-null   int64  
 2   MSSubClass     2909 non-null   int64  
 3   MSZoning       2909 non-null   object 
 4   LotFrontage    2909 non-null   float64
 5   LotArea        2909 non-null   int64  
 6   Street         2909 non-null   object 
 7   Alley          2909 non-null   object 
 8   LotShape       2909 non-null   object 
 9   LandContour    2909 non-null   object 
 10  Utilities      2909 non-null   object 
 11  LotConfig      2909 non-null   object 
 12  LandSlope      2909 non-null   object 
 13  Neighborhood   2909 non-null   object 
 14  Condition1     2909 non-null   object 
 15  Condition2     2909 non-null   object 
 16  BldgType       2909 non-null   object 
 17  HouseStyle     2909 non-null   object 
 18  OverallQual  

In [35]:
df["SaleType"].value_counts()

SaleType
WD       2516
New       239
COD        87
ConLD      25
CWD        12
ConLI       9
ConLw       8
Oth         7
Con         5
Name: count, dtype: int64

In [36]:
# Filling using WD since it's the most common
df.fillna({"SaleType": "WD"}, inplace=True)

Since the testing set does not contain the sale price we really cannot use this combined dataset. Since both datasets are now cleaned, it makes it easier to just separate the datasets from one another and use them.

In [37]:
training = df.drop(index=df[df["SalePrice"].isna()].index)
testing = df.drop(index=df[df["SalePrice"].notna()].index, columns="SalePrice")

# What now

Now that the data has been cleaned let's save these datasets and proceed with exploration to get a better idea of what's going on.

In [38]:
p_dir = '../data/cleaned/'

training.to_csv(p_dir + "training.csv")
testing.to_csv(p_dir + "testing.csv")